# Homework 3 - Application Building

- Python Computing for Data Science (2022)

- Due Tuesday Feb 15 (8pm)

## CalCalc

Write a module called `CalCalc`, with a method called `calculate` that evaluates any string passed to it, and can be used from either the command line (using `argparse` with reasonable flags) or imported within Python. Feel free to use something like `eval()`, but be aware of some of the nasty things it can do, and make sure it doesn’t have too much power:  http://nedbatchelder.com/blog/201206/eval_really_is_dangerous.html. Perhaps explore the use of `numexpr` to constrain the landscape of possible uses to math expressions.

EXAMPLE:
```bash
$ python CalCalc.py -s '34*28'
$ 952
```
 AND, from within Python
 
```python
>>> from CalCalc import calculate
>>> calculate('34*20')
>>> 952
```

Eventually I'll merge the base calculate from this section with the wolfram update. Untill I get that working, I'll keep a separate copy of the original 1.1 ...

In [1]:
%%writefile CalCalc_part1.py
"""CalCalc module with some nice methods to evaluate strings (as
numerical expressions only).
"""

import argparse
import numexpr as ne


def calculate(str_input):
    """Evaluate any string, limited to numerical expressions.

    Parameters
    ----------
    str_input : str
        String expression to evaluate.

    Returns
    -------
    answer : float
        Numeric output of evaluated string.

    """

    # raise error if 'str_input' is not string
    if type(str_input) != str:
        raise ValueError("Did not receive string.")

    # attempt to evaluate 'str_input' using numexpr
    try:
        answer = ne.evaluate(str_input)
        # TODO: answer is np array, kinda ugly... come back to fix
        return answer

    except KeyError:
        raise ValueError("Could not evaluate with numexpr.")


if __name__ == '__main__':
    # parse command line arguements
    parser = argparse.ArgumentParser(description='CalCal Module')
    parser.add_argument('-s', action='store', dest='str_input',
                        help='String to parse')
    results = parser.parse_args()

    # TODO: try using click instead to make cuter!

    # execute function
    print(calculate(results.str_input))

Overwriting CalCalc_part1.py


In [2]:
!flake8 CalCalc_part1.py # looks good!

In [3]:
!python CalCalc_part1.py -s '34*28'

952


In [4]:
from CalCalc_part1 import calculate
calculate('34*28')

array(952, dtype=int32)

### Add Wolfram

To make this more awesome, have your function interact with the Wolfram|Alpha API to ask it what it thinks of the difficult questions.  To make this work, experiment with `urllib2` and a URL like this:
'http://api.wolframalpha.com/v2/query?input=XXXXX&appid=UAGAWR-3X6Y8W777Q'
where you replace the XXXXX with what you want to know.  NOTE: the ‘&appid=UAGAWR-3X6Y8W777Q’ part is vital; it is a W|A AppID I got for the class.  Feel free to use that one, or you can get your own and read more about the API, here:   http://products.wolframalpha.com/api/
And you can explore how it works here:  http://products.wolframalpha.com/api/explorer.html

EXAMPLE:

```bash
$ python CalCalc.py -w 'mass of the moon in kg'
7.3459e+22
```

AND, from within Python

```python
>>> from CalCalc import calculate
>>> calculate('mass of the moon in kg',  return_float=True) * 10
>>> 7.3459e+23
```

In [105]:
# figure out how to interact with wolfram first...
import urllib
import numexpr as ne

# put together query address
url_prefix = 'http://api.wolframalpha.com/v2/query?input=' 
query = 'mass of the moon in kg'
query_nice = query.replace('+','%2B').replace(' ','+') # replace numeric '+' symbols; make '+' = space;
query_opts = '&appid=9TLK2V-8T9H43UY82&format=plaintext&includepodid=Result' # reduce output to just results as text

full_url = url_prefix + query_nice + query_opts
f = urllib.request.urlopen(full_url)
f_as_text = f.read().decode('utf-8')
# print(f_as_text)

# pull out answer from plaintext section of query output
idx_0 = f_as_text.find('<plaintext>')+11
idx_stop = f_as_text.find('</plaintext>',idx_0)
print(f_as_text[idx_0:idx_stop])

# turn the first thing into a number
num = f_as_text[idx_0:idx_stop].split(' ')[0]
ne.evaluate(num.replace('×','*').replace('^','**'))


7.3459×10^22 kg (kilograms)


array(7.3459e+22)

In [110]:
# let's make the fancy function!

In [140]:
%%writefile CalCalc.py
"""CalCalc: evaluate strings as numeric expressions. If simple (i.e., no 
alpha characters), will try to evaluate locally with numexpr. If more 
complex, will try to query wolfram alpha.
"""

import argparse
import numexpr as ne
import urllib.request


def calculate(str_input, return_float=True):
    """Evaluate any string, limited to numerical expressions.

    Parameters
    ----------
    str_input : str
        String expression to evaluate.
    return_float : bool
        If output should be float (default = True)

    Returns
    -------
    answer : float
        Numeric output of evaluated string.

    """
    
    # first, are you string?
    if type(str_input) != str:
        raise ValueError("You are not a string. Try again.")
    
    # let's see if numexpr can solve you
    try: 
        answer = ne.evaluate(str_input).item()
        return answer 
    except (KeyError, SyntaxError) as e: 
        
        # ok, let's send you to wolfram
        try: 
            answer = query_wolfram(str_input, return_float)
            return answer
        except (ValueError, SyntaxError) as e:
            print("Wolfram could not solve query.")
        
        
    
def query_wolfram(str_input, return_float):
    """Send off query to wolfram...
    
    Parameters
    ----------
    str_input : str
        String expression to evaluate, e.g. 'mass of moon in kg'
    return_float : bool
        If output should be float (default = True)

    Returns
    -------
    query_output : str or float
        Full text restult or just float, depending on 'return_float'

    """

    # get query url
    url_prefix = 'http://api.wolframalpha.com/v2/query?input=' 
    query = str_input.replace('+', '%2B').replace(' ','+') # replace numeric '+'; use '+' as space;
    return_opts = '&appid=9TLK2V-8T9H43UY82&format=plaintext&includepodid=Result' # reduce output to just results as text

    full_url = url_prefix + query + return_opts

    # set off to wolfram!
    output = urllib.request.urlopen(full_url) # request for url
    output_as_text = output.read().decode('utf-8') # turn it into English...

    # extract answer: for now, just do brute force and find the plaintext tags in the results pod
    # if time, check out beautiful soup?
    idx_plaintext = output_as_text.find('<plaintext>')
    
    if idx_plaintext == -1:
        raise ValueError("Don't know how to parse result from wolfram.")
    
    idx_start = idx_plaintext + 11
    idx_stop = output_as_text.find('</plaintext>',idx_start)
    query_output = output_as_text[idx_start:idx_stop]
    
    if query_output == '(data not available)':
        return ValueError('Wolfram does not know the answer.')
    
    if return_float:
        number = query_output.split(' ')[0]
        number_simple = number.replace('×','*').replace('^','**')
        return ne.evaluate(number_simple).item()
    
    else: 
        return query_output
    
    
if __name__ == '__main__':
    # parse command line arguements
    parser = argparse.ArgumentParser(description='CalCal Module')
    parser.add_argument('-s', action='store', dest='str_input',
                        help='String to parse') # use the same flag for all inputs! 
    results = parser.parse_args()

    # TODO: try using click instead to make cuter!

    # check you got an arg
    if results.str_input == None:
        raise ValueError('feed me a string with -s')
    
    # execute function
    print(calculate(results.str_input))
    

Overwriting CalCalc.py


In [141]:
!python CalCalc.py -s '34*28'

952


In [144]:
!python CalCalc.py -s 'mass of the moon in kg'

7.3459e+22


In [150]:
from CalCalc import calculate


In [153]:
calculate('34*4')


array(136, dtype=int32)

In [156]:
x = calculate('mass of the moon in kg')

## Adding it to Github

Start a github project for CalCalc. Include a setup.py, README.txt, LICENSE.txt, MANIFEST.in, etc. and turn your module into a proper Python Distribution, so that we can install it and use it. See https://packaging.python.org/en/latest/guides/distributing-packages-using-setuptools/ 

Example Folder Hierarchy:
```bash
Your_Homework3_Folder/calcalc
                      |--> CalCalc.py
                      |--> __init__.py
Your_Homework3_Folder/setup.py
Your_Homework3_Folder/README.txt
...
```
Include at least 5 test functions in CalCalc.py, and test with `pytest`, to make sure it behaves the way you think it should.

EXAMPLE `CalCalc.py`:
```python
# ...
def calculate([...]):
    [...]

def test_1():
    assert abs(4. - calculate('2**2')) < 0.001
```

When grading, we will create a virtual environment and attempt to install your module by running:

```bash
pip install build
```

### CalCalc on CI

Get your project working with GitHub Actions and make sure your tests are run and pass. Give us a link to you GH actions for your site here (e.g. https://github.com/profjsb/PyAdder/actions):

### **(Bonus/Extra Credit)** 

  Get your project working on Azure, AWS or Google Compute Cloud with a Flask front-end. You can use the example from class as a template. Start a VM on one of these PaaS. A user should be able to submit their calcalc query on a form (hosted on your VM) and get the result back.

You should be able to add an `app.py` (with Flask) into your CalCalc project. Be sure to open up the port on the VM that you are serving on. Let us know the URL to your app here: